In [1]:
import os
import pandas as pd 
import numpy as np
from my_translation.my_translation_utils import help_to_ukrainian
import warnings
import re

warnings.filterwarnings('ignore')

path = os.path.join(os.getcwd(), 'DATA/dates.txt')
doc = []
with open(path) as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
df.shape[0]

500

In [3]:
a1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
a1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
a1 = pd.concat([a1_1, a1_2])
a1.reset_index(inplace=True)
a1 = a1.rename(columns={'level_0': 'Index', 0: 'Month', 1: 'Day', 2: 'Year'})
final_a1 = a1[['Index', 'Month', 'Day', 'Year']]
final_a1.Month = final_a1.Month.apply(lambda x: x if len(x) == 2 else '0' + x)
final_a1.Day = final_a1.Day.apply(lambda x: x if len(x) == 2 else '0' + x)
final_a1.Year = final_a1.Year.apply(lambda x: x if len(x) == 4 else '19' + x)
final_a1

,Index,Month,Day,Year
0,0,03,25,1993
1,1,06,18,1985
2,2,07,08,1971
3,3,09,27,1975
4,4,02,06,1996
...,...,...,...,...
120,95,04,11,1974
121,102,07,18,2002
122,106,02,11,2006
123,109,07,20,2011


In [4]:
a1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Index   125 non-null    int64 
 1   match   125 non-null    int64 
 2   Month   125 non-null    object
 3   Day     125 non-null    object
 4   Year    125 non-null    object
dtypes: int64(2), object(3)
memory usage: 5.0+ KB


In [5]:
df1 = df.drop(index=final_a1['Index'])

In [6]:
df1

125    s The patient is a 44 year old married Caucasi...
126    .10 Sep 2004 - Intake at EEC for IOP but did n...
127    see above and APS eval of 26 May 1982 Social H...
128    Tbooked for intake appointment at Sierra Vista...
129                    06 May 1972 SOS-10 Total Score:\n
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 375, dtype: object

In [7]:
df1.shape

(375,)

In [8]:
a2 = df1.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
a2.reset_index(inplace=True)
a2_1 = a2[1].str.extractall(r'((\d{1,2},? )(\d{2,4}))')
a2_1[1] = a2_1[1].apply(lambda x: x[:2])
a2.index = a2['level_0']
a2['Day'] = a2_1.values[:,1]
a2['Year'] = a2_1.values[:,2]
a2['Index'] = a2.index
final_a2 = pd.DataFrame({
    'Index': a2['level_0'],
    'Month': a2[0],
    'Day': a2['Day'],
    'Year': a2['Year']
})
final_a2.reset_index(drop=True, inplace=True)
final_a2

,Index,Month,Day,Year
0,194,April,11,1990
1,195,May,30,2001
2,196,Feb,18,1994
3,197,February,18,1981
4,198,October.,11,2013
5,199,Jan,24,1986
6,200,July,26,1978
7,201,December,23,1999
8,202,May,15,1989
9,203,September,06,1995


In [9]:
a2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 194 to 227
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   level_0  34 non-null     int64 
 1   match    34 non-null     int64 
 2   0        34 non-null     object
 3   1        34 non-null     object
 4   Day      34 non-null     object
 5   Year     34 non-null     object
 6   Index    34 non-null     int64 
dtypes: int64(3), object(4)
memory usage: 2.1+ KB


In [10]:
df2 = df1.drop(index=final_a2['Index'])
df2

125    s The patient is a 44 year old married Caucasi...
126    .10 Sep 2004 - Intake at EEC for IOP but did n...
127    see above and APS eval of 26 May 1982 Social H...
128    Tbooked for intake appointment at Sierra Vista...
129                    06 May 1972 SOS-10 Total Score:\n
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 341, dtype: object

In [11]:
df2.shape

(341,)

In [12]:
a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
a3[0].fillna('01', inplace=True)
index = [x[0] for x in a3.index]
a3['Index'] = index
a3['Month'] = a3[1].str.extract(r'([a-zA-Z]*)')
a3['Day'] = a3[0].apply(lambda x: x[:2])
a3['Year'] = a3[2]
final_a3 = pd.DataFrame({
    'Index': a3['Index'],
    'Month': a3['Month'],
    'Day': a3['Day'],
    'Year': a3['Year']
})
final_a3.reset_index(drop=True, inplace=True)
final_a3

,Index,Month,Day,Year
0,125,Jan,24,2001
1,126,Sep,10,2004
2,127,May,26,1982
3,128,June,28,2002
4,129,May,06,1972
...,...,...,...,...
179,338,Apr,01,1998
180,339,March,01,2005
181,340,May,01,1980
182,341,Nov,01,2007


In [13]:
final_a3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Index   184 non-null    int64 
 1   Month   184 non-null    object
 2   Day     184 non-null    object
 3   Year    184 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.9+ KB


In [14]:
df3 = df2.drop(index=final_a3['Index'])
df3

343                        6/1998 Primary Care Doctor:\n
344    s 52 y/o MWM h/o chronic depression, anxiety, ...
345                    10/1973 Hx of Brain Injury: Yes\n
346                        9/2005 Primary Care Doctor:\n
347    s 03/1980 Positive PPD: treated with INH for 6...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                         2008 partial thyroidectomy\n
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 157, dtype: object

In [15]:
df3.shape

(157,)

In [16]:
a4 = df3.str.extractall(r'(\d{1,2})[/](\d{4})')
index = [x[0] for x in a4.index]
a4['Index'] = index
a4['Month'] = a4[0].apply(lambda x: x if len(x) == 2 else '0' + x)
a4['Day'] = '01'
a4['Year'] = a4[1]
final_a4 = pd.DataFrame({
    'Index': a4['Index'],
    'Month': a4['Month'],
    'Day': a4['Day'],
    'Year': a4['Year']
})
final_a4.reset_index(drop=True, inplace=True)
final_a4

,Index,Month,Day,Year
0,343,06,01,1998
1,344,06,01,2005
2,345,10,01,1973
3,346,09,01,2005
4,347,03,01,1980
...,...,...,...,...
107,450,01,01,1994
108,451,12,01,2004
109,452,03,01,2003
110,453,07,01,1991


In [17]:
final_a4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Index   112 non-null    int64 
 1   Month   112 non-null    object
 2   Day     112 non-null    object
 3   Year    112 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.6+ KB


In [18]:
df4 = df3.drop(index=final_a4['Index'])
df4

455    sHemmorage caused by probe in 1984 Medical His...
456    sHas been at MYH since his treaters in NE reti...
457    Pt joined Army reserves in 2001 and has 3 year...
458    one sister from whom he is estranged due to he...
459    sSince 1998. Prior medication trials (includin...
460                            1 Ex-smoker : quit 2012\n
461    . Age 16, 1991, frontal impact. out for two we...
462       sLexapro (1988-now): Good response (anxiety)\n
463    s      25 year old engaged to be married Optic...
464    8Complications from brain hemmorage in 2016 Ax...
465    .Age, 19, 1976, playing football, frontal impa...
466                   s1981  Swedish-American Hospital\n
467    aS/P suicide attempt 2011 Hx of Outpatient Tre...
468    Patient has a history of suicidal ideation wit...
469    Born and raised in Fowlerville, IN.  Parents d...
470    y1983 Clinic Hospital, first hospitalization, ...
471    tProblems Urinary incontinence : mild urge inc...
472    .2010 - wife; nightmares

In [19]:
df4.shape

(45,)

In [20]:
a5 = df4.str.extractall(r'(\d{4})')

index = [x[0] for x in a5.index]
a5['Index'] = index
a5['Month'] = '01'
a5['Day'] = '01'
a5['Year'] = a5[0]
final_a5 = pd.DataFrame({
    'Index': a5['Index'],
    'Month': a5['Month'],
    'Day': a5['Day'],
    'Year': a5['Year']
})
final_a5.reset_index(drop=True, inplace=True)
final_a5

,Index,Month,Day,Year
0,455,01,01,1984
1,456,01,01,2000
2,457,01,01,2001
3,458,01,01,1982
4,459,01,01,1998
5,460,01,01,2012
6,461,01,01,1991
7,462,01,01,1988
8,463,01,01,2014
9,464,01,01,2016


In [21]:
df_final = pd.concat([final_a1, final_a2, final_a3, final_a4, final_a5])
df_final

,Index,Month,Day,Year
0,0,03,25,1993
1,1,06,18,1985
2,2,07,08,1971
3,3,09,27,1975
4,4,02,06,1996
...,...,...,...,...
40,495,01,01,1979
41,496,01,01,2006
42,497,01,01,2008
43,498,01,01,2005


In [22]:
df_final['Month'] = df_final['Month'].apply(lambda x: x[:-1] if x[-1] == ' ' else x)
df_final['Month'] = df_final['Month'].apply(lambda x: x[:-1] if x[-1] == '.' else x)
df_final['Month'].unique()

array(['03', '06', '07', '09', '02', '05', '10', '04', '08', '01', '11',
       '12', 'April', 'May', 'Feb', 'February', 'October', 'Jan', 'July',
       'December', 'September', 'Mar', 'August', 'Sep', 'Nov', 'June',
       'Dec', 'Oct', 'January', 'March', 'Aug', 'Jun', 'Jul', 'November',
       'Apr', 'Janaury', 'Decemeber'], dtype=object)

In [23]:
month_mapping = {
    'Jan': '01', 'January': '01', 'Janaury': '01', 'Feb': '02', 'February': '02', 'Mar': '03', 'March': '03', 'Apr': '04', 'April': '04',
    'May': '05', 'Jun': '06', 'June': '06', 'Jul': '07', 'July': '07', 'Aug': '08', 'August': '08', 'Sep': '09', 'September': '09',
    'Oct': '10', 'October': '10', 'Nov': '11', 'November': '11', 'Dec': '12', 'December': '12', 'Decemeber': '12'
}

df_final['Month'] = df_final['Month'].replace(month_mapping)
df_final['Month'].unique()

array(['03', '06', '07', '09', '02', '05', '10', '04', '08', '01', '11',
       '12'], dtype=object)

In [24]:
df_final['Date'] = pd.to_datetime(df_final['Month'].astype(str) + '/' + df_final['Day'].astype(str) + '/' + df_final['Year'].astype(str), errors='coerce')
df_final['Month'].unique()

array(['03', '06', '07', '09', '02', '05', '10', '04', '08', '01', '11',
       '12'], dtype=object)

In [25]:
df_final.sort_values(['Date'], inplace=True)
df_final.reset_index(inplace=True)
df_final = df_final[['Index', 'Date']]
df_final

,Index,Date
0,9,1971-04-10
1,84,1971-05-18
2,2,1971-07-08
3,53,1971-07-11
4,28,1971-09-12
...,...,...
495,427,2016-05-01
496,141,2016-05-30
497,186,2016-10-13
498,161,2016-10-19
